In [1]:
import numpy as np
import os
import copy

import open3d as o3d
from open3d import JVisualizer

In [2]:
class Mesh:
    def __init__(self, filepath):
        os.path.isfile(filepath)
        self.mesh_o3d = o3d.io.read_triangle_mesh(filepath)
        self.print_mesh()

    def visualize(self):
        self.mesh_o3d.compute_vertex_normals()
        vis = o3d.visualization.Visualizer()
        vis.create_window()
        vis.get_render_option().mesh_show_back_face = True
        self.add_to_vis(vis)
        mesh_frame = o3d.geometry.create_mesh_coordinate_frame(size=4,
                                                               origin=[0, 0, 0])
        vis.add_geometry(mesh_frame)
        vis.run()
        vis.destroy_window()

    def add_to_vis(self, vis):
        """ Add the mesh to the visualization `vis` object """
        self.mesh_o3d.compute_vertex_normals()
        vis.add_geometry(self.mesh_o3d)

    def print_mesh(self):
        print("Testing mesh in open3d ...")
        print(self.mesh_o3d)
        print(np.asarray(self.mesh_o3d.vertices))
        print(np.asarray(self.mesh_o3d.triangles))
        print("")
    
    def transform(self, T):
        self.mesh_o3d.transform(T)

In [3]:
def quaternion_to_matrix(q):
    # q: numpy array: [x, y, z, w]
    x = q[0]
    y = q[1]
    z = q[2]
    w = q[3]
    Nq = w*w + x*x + y*y + z*z
    if Nq < np.finfo(np.float).eps:
        return np.eye(3)
    s = 2.0/Nq
    X = x*s
    Y = y*s
    Z = z*s
    wX = w*X; wY = w*Y; wZ = w*Z
    xX = x*X; xY = x*Y; xZ = x*Z
    yY = y*Y; yZ = y*Z; zZ = z*Z
    return np.array(
           [[ 1.0-(yY+zZ), xY-wZ, xZ+wY ],
            [ xY+wZ, 1.0-(xX+zZ), yZ-wX ],
            [ xZ-wY, yZ+wX, 1.0-(xX+yY) ]])

In [144]:
# FILL PATHS BELOW
gt_mesh_path = "/home/yunchang/Desktop/dcist_gt_mesh/camp/camp_gt.ply"
est_mesh_path = "/home/yunchang/Downloads/tro_results/camp/mesh_pgmo.ply"

In [145]:
# FILL IN 
robot_0_position_0 = np.array([0, 0, 0])
robot_0_quaternion_0 = np.array([0, 0, 0, 1])

In [146]:
print("Loading Ground-truth mesh...")
gt_mesh_original = Mesh(gt_mesh_path)
print("Loading Estimated mesh...")
est_mesh_original = Mesh(est_mesh_path)

Loading Ground-truth mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 35706535 points and 37947294 triangles.
[[-1.99783e+01 -5.85750e+01  2.50000e-02]
 [-1.99750e+01 -5.85934e+01  2.50000e-02]
 [-1.99750e+01 -5.85750e+01  2.37550e-02]
 ...
 [ 6.17500e+00 -3.01750e+01  2.41304e+00]
 [ 6.17667e+00 -3.01750e+01  2.42500e+00]
 [ 6.19762e+00 -3.01250e+01  2.42500e+00]]
[[       0        1        2]
 [       3        0        2]
 [       4        5        6]
 ...
 [35706361 35706360 35706532]
 [35706533 35706361 35706532]
 [35706534 35706361 35706533]]

Loading Estimated mesh...
Testing mesh in open3d ...
geometry::TriangleMesh with 2930323 points and 11545216 triangles.
[[ 75.5388  -49.1382    6.2504 ]
 [ 75.6995  -49.0844    6.2566 ]
 [ 75.5686  -48.9804    6.23622]
 ...
 [-39.4781   -9.96037   2.56177]
 [-39.5347   -8.9882    2.50582]
 [-41.1905   -7.34573   2.42424]]
[[      0       1       2]
 [      3       1       0]
 [      4       3       0]
 ...
 [2928586 2928591 292

In [147]:
# Transform Meshes to same frame of reference
gt_mesh = copy.deepcopy(gt_mesh_original)
est_mesh = copy.deepcopy(est_mesh_original)

In [148]:
# Transform gt mesh to est mesh 
w_T_gtPose0 = np.eye(4)
w_T_gtPose0[:3, :3] = quaternion_to_matrix(robot_0_quaternion_0)
w_T_gtPose0[:3, 3] = robot_0_position_0

w_T_estPose0 = np.eye(4) # first pose of first robot after dpgo is identity 

est_T_gt = np.dot(np.linalg.inv(w_T_gtPose0), w_T_estPose0)
gt_mesh.transform(est_T_gt)

In [149]:
# vis = o3d.visualization.Visualizer()
# vis.create_window()
# vis.get_render_option().mesh_show_back_face = True
# vis.add_geometry(est_mesh.mesh_o3d)
# vis.add_geometry(gt_mesh.mesh_o3d)
# # vis.add_geometry(o3d.geometry.create_mesh_coordinate_frame(size=4))
# vis.run()
# vis.destroy_window()

In [150]:
NUMBER_OF_SAMPLES=1000000
gt_pcl = gt_mesh.mesh_o3d.sample_points_uniformly(NUMBER_OF_SAMPLES)
# Don't sample estimated mesh, just pick vertices, otw you'll be mixing colors...
# est_pcl = o3d.geometry.sample_points_uniformly(est_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
est_pcl = o3d.io.read_point_cloud(est_mesh_path)

In [151]:
# vis = o3d.visualization.Visualizer()
# vis.create_window()
# vis.get_render_option().mesh_show_back_face = True
# vis.add_geometry(gt_pcl)
# vis.add_geometry(est_pcl)
# vis.run()
# vis.destroy_window()

In [152]:
# ICP
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
def draw_correspondences(source, target, correspondences):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, #target_temp, 
                                       correspondences])
    

In [153]:
# ICP params
ICP_THRESHOLD = 1.0
trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0],
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0],
                         [0.0, 0.0, 0.0, 1.0]])

In [154]:
# # Visualize initial registration problem
# draw_registration_result(est_pcl, gt_pcl, trans_init)

In [155]:
# Evaluate current fit between pointclouds
evaluation = o3d.registration.evaluate_registration(est_pcl, gt_pcl, ICP_THRESHOLD, trans_init)

In [156]:
print("Initial registration")
print(evaluation)

Initial registration
registration::RegistrationResult with fitness=1.391884e-01, inlier_rmse=6.023816e-01, and correspondence_set size of 407867
Access transformation to get result.


In [157]:
print("Apply point-to-point ICP")
reg_p2p = o3d.registration.registration_icp(
    est_pcl, gt_pcl, ICP_THRESHOLD, trans_init,
    o3d.registration.TransformationEstimationPointToPoint(),
    o3d.registration.ICPConvergenceCriteria(max_iteration = 2000))
correspondences = reg_p2p.correspondence_set

Apply point-to-point ICP


In [158]:
print(reg_p2p)
print("")

print("Transformation is:")
print(reg_p2p.transformation)
print("")

print("Correspondence Set:")
print(reg_p2p.correspondence_set)
print("")

registration::RegistrationResult with fitness=5.401708e-01, inlier_rmse=4.780114e-01, and correspondence_set size of 1582875
Access transformation to get result.

Transformation is:
[[ 0.99437222  0.09242255  0.05178764  6.41835053]
 [-0.09020172  0.99496435 -0.04369879  9.36272342]
 [-0.05556561  0.03878153  0.99770159 -0.40419754]
 [ 0.          0.          0.          1.        ]]

Correspondence Set:
std::vector<Eigen::Vector2i> with 1582875 elements.
Use numpy.asarray() to access data.



In [159]:
# # Draw Registration Result
# draw_registration_result(est_pcl, gt_pcl, reg_p2p.transformation)

In [160]:
# # Draw Only Correspondences
# c2c_lines = o3d.geometry.create_line_set_from_point_cloud_correspondences(est_pcl, gt_pcl, correspondences)
# o3d.visualization.draw_geometries([c2c_lines])

In [161]:
# # Draw PointClouds and Correspondences
# draw_correspondences(est_pcl, gt_pcl, c2c_lines)

In [162]:
def calc_corresp(est_pcl, gt_pcl, correspondences):
    total_negative_matches = 0
    total_positive_matches = 0
    total_correspondences = len(correspondences)
    for correspondence in correspondences:
        if np.allclose(est_pcl.colors[correspondence[0]],
                       gt_pcl.colors[correspondence[1]]):
            total_positive_matches += 1
        else:
            total_negative_matches += 1

    print("Positive color matches: ",total_positive_matches)
    print("Negative color matches: ", total_negative_matches)
    print("Total correspondences: ", total_correspondences)
    assert(total_correspondences == total_negative_matches + total_positive_matches)
    print ("Positive: {}  % ".format(float(total_positive_matches) / float(total_correspondences) * 100.0))
    print ("Negative: {}  % ".format(float(total_negative_matches) / float(total_correspondences) * 100.0))

In [163]:
calc_corresp(est_pcl, gt_pcl, correspondences)

('Positive color matches: ', 1257472)
('Negative color matches: ', 325403)
('Total correspondences: ', 1582875)
Positive: 79.4422806602  % 
Negative: 20.5577193398  % 
